# Trabalho Prático 2

## Introdução

A resolução deste trabalho prático tem como propósito introduzir o **SageMath**, corpos finitos primos, curvas elípticas sobre esses corpos e esquemas criptográficos baseados nos mesmos.
Os principais objetivos passam por:

- Implementar um esquema _RSA_ como uma classe **Python**.
- Implementar um esquema _ECDSA_ como uma classe **Python**.
- Implementar um esquema _ECDH_ como uma classe **Python**.

Além disso, devem ser aplicadas as seguintes particularidades aos esquemas desenvolvidos:

- O esquema _RSA_ deve fornecer métodos para cifrar, decifrar, assinar e verificar uma mensagem.
- O esquema _ECDSA_ deve utilizar uma das curvas primas definidas no **FIPS186-4**.
- O esquema _ECDH_ deve utilizar curvas elíticas binárias.

O relatório está dividido em três partes, uma por cada um dos objetivos a cumprir. Além disso, está estruturado de forma a que o texto entre os _snippets_ de código seja suficientemente explicativo, sobre a implementação e desenho da solução.

## Imports e funções comuns

Esta secção tem como objetivo importar as bibliotecas que serão necessárias na definição dos esquemas implementados neste trabalho, bem como qualquer função que seja comum aos mesmos e que possa ser utilizada em dois ou mais casos.

In [1]:
import hashlib
from sage.crypto.util import ascii_to_bin, bin_to_ascii

def hash_message(message):
    digest = hashlib.sha256(message).hexdigest()
    return digest

def convert_to_ZZ(message):
    raw = ascii_to_bin(message)
    return ZZ(int(str(raw),2))

def bpf(factors):
    f = 0
    for pair in factors:
        p = pair[0]
        if p > f:
            f = p
    return f
    

## Esquema RSA

O objetivo desta secção passa por definir a classe **Python** que implementa o algoritmo **RSA**. Esta permitirá inicializar uma intância, fornecendo-lhe o parâmetro de segurança (tamanho de chave). Após criada a instância, a mesma permitirá:

- Cifrar uma mensagem.
- Decifrar uma mensagem previamente cifrada.
- Assinar digitalmente uma mensagem.
- Verificar uma assinatura digital de uma mensagem, previamente produzida.

### Definição do esquema RSA

In [2]:
class RSA:
    
    def __init__(self,l):
        while True:
            r = random_prime(2**l-1,True,2**(l-1))
            p = random_prime(2**(l+1),True, 2**l)
            if 2*r >= 2**(l/2) and p > 2*r:
                break
        self.q = p * r
        phi = (p - 1) * (r - 1)
        
        k = ZZ.random_element(phi)
        while gcd(k, phi) != 1:
             k = ZZ.random_element(phi)
        self.public_key = k
        
        self.private_key = inverse_mod(self.public_key,phi)
    
    def encrypt(self,plaintext):
        plaintext = convert_to_ZZ(plaintext)
        ciphertext_zz = power_mod(plaintext,self.public_key,self.q)
        ciphertext_zz_raw = ciphertext_zz.binary()
        
        bits_missing = 8 - Mod(len(ciphertext_zz_raw),8)
        raw = ('0' * bits_missing) + ciphertext_zz_raw
        ciphertext = bin_to_ascii(raw)
        
        return ciphertext      
    
    def decrypt(self,ciphertext):
        ciphertext = convert_to_ZZ(ciphertext)
        plaintext = power_mod(ciphertext,self.private_key,self.q)
        raw_b = plaintext.binary()
        
        bits_missing = 8 - Mod(len(raw_b),8)
        raw = ('0' * bits_missing) + raw_b

        return bin_to_ascii(raw)        
    
    def sign(self,message):
        msg_hash = hash_message(message)
        message = convert_to_ZZ(msg_hash)
        return power_mod(message,self.private_key,self.q)        
    
    def verify(self,message,signature):
        mk = power_mod(signature,self.public_key,self.q)
        mk_raw = mk.binary()
        
        bits_missing = 8 - Mod(len(mk_raw),8)
        mk_raw = ('0' * bits_missing) + mk_raw
        
        message = hash_message(message)
        mk_raw = bin_to_ascii(mk_raw)
        return mk_raw == message

### Teste ao Esquema RSA

In [3]:
# Iniciar uma instância RSA com parâmetro de segurança de 1024.
r = RSA(1024)
msg = "Estruturas Criptográficas - Trabalho 2 - Iniciação SageMath - Esquema RSA"
print('message:')
print(msg)

#cifrar a mensagem com a instância previamente criada
ciphertext = r.encrypt(msg)

print('\n ciphertext:')
print(ciphertext)
print('\n')

#Decifrar o criptograma previamente produzido
plaintext = r.decrypt(ciphertext)
print('decrypted:')
print(plaintext)

#Assinar e verificar a assinatura
print('\n')
print('signature result:')
sig = r.sign(msg)
if(r.verify(msg,sig)):
    print('OK')
else: print('Not OK!')

message:
Estruturas Criptográficas - Trabalho 2 - Iniciação SageMath - Esquema RSA

 ciphertext:
)�р�^ܢ�!��Ȧv�F��s��5������Ȃ����1b� s	h)W����j�D����y(6h0�iױ�՛�[x���o� �f�J����Ȭ0�$h�>�ٞ���  ES����l����\�=�����B�f�t[#��������Z�����8�bpD�lh##�6ȓ���|?1Nү%�g���b���Y1 ���dٽ�v��Xw�����p�V&���%�����n�ɦ��$��*kO�v


decrypted:
Estruturas Criptográficas - Trabalho 2 - Iniciação SageMath - Esquema RSA


signature result:
OK


## Esquema ECDSA

### Definição da curva _NIST_

In [4]:
NIST = dict()
NIST['P-256'] = {
    'p': 115792089210356248762697446949407573530086143415290314195533631308867097853951,
    'n': 115792089210356248762697446949407573529996955224135760342422259061068512044369,
    'seed': 'c49d360886e704936a6678e1139d26b7819f7e90',
    'c': '7efba1662985be9403cb055c75d4f7e0ce8d84a9c5114abcaf3177680104fa0d',
    'b': '5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b',
    'Gx': '6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296',
    'Gy': '4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5'
}

### Definição do esquema ECDSA

In [5]:
class ECDSA:
    
    def __init__(self):
        curve = NIST['P-256']
        p = curve['p']
        self.n = curve['n']
        b = ZZ(curve['b'],16)
        Gx = ZZ(curve['Gx'],16)
        Gy = ZZ(curve['Gy'],16)
        self.E = EllipticCurve(GF(p),[-3,b])
        self.G = self.E((Gx,Gy))
        self.private_key = ZZ.random_element(1,self.n - 1)
        self.public_key = self.private_key * self.G
    
    
    def sign(self,message):
        digest = hash_message(message)
        digest = convert_to_ZZ(digest)
        back_to_1 = False
        while not back_to_1:
            ok = False
            k = ZZ.random_element(1,self.n-1)
            r_point = k * self.G
            r = Mod(r_point[0],self.n)
            if r == 0:
                back_to_1 = False
            else :
                while not ok:
                    k_inverse = inverse_mod(k,self.n)
                    temp_calc = k_inverse * (digest + (r*self.private_key))
                    s = ZZ(Mod(temp_calc,self.n))
                    if s == 0:
                        ok = True
                    else:
                        ok = True
                        back_to_1 = True
        
        return r,s
    
    def verify(self,message,signature):
        sig_r = signature[0]
        sig_s = signature[1]
        if (sig_r < 1 or sig_r > self.n -1 or sig_s < 1 or sig_s > self.n - 1):
            return False
        else:
            digest = hash_message(message)
            digest = convert_to_ZZ(digest)
            w = inverse_mod(sig_s,self.n)
            u1 = ZZ(Mod(digest*w,self.n))
            u2 = ZZ(Mod(sig_r*w,self.n))
            cp = u1*self.G + u2*self.public_key
            if Mod(cp[0],self.n) == Mod(sig_r,self.n):
                return True
            else:
                return False

### Teste ao esquema ECDSA

In [6]:
e = ECDSA()
msg = "Estruturas Criptográficas - Trabalho 2 - Iniciação SageMath - Esquema RSA"
r,s = e.sign(msg)
if e.verify(msg,(r,s)):
    print 'OK'
else:
    print 'Not OK'

OK


## Esquema ECDH

### Definição do esquema ECDH

In [7]:
class ECDH:
    
    def __init__(self,n):
        K.<t> = GF(2^n)
        ok = False
        while not ok:
            b = K.random_element()
            E = EllipticCurve(K,[1,1,0,0,b])
            e_order = E.order()
            F = factor(e_order)
            N = bpf(list(F))
            if N < 2^(n-1):
                pass
            else:
                while True:
                    P = E.random_point()
                    P_order = P.order()
                    if P_order < N:
                        pass
                    elif P_order > N:
                        h = ZZ(P_order/N)
                        self.G = h*P
                        self.N = N
                        self.b = b
                        ok = True
                        break
                    else:
                        self.N = N
                        self.b = b
                        self.G = P
                        ok = True
                        break
        
    def generate_key_pair(self):
        private_key = ZZ.random_element(1,self.N-1)
        public_key = private_key * self.G
        return (private_key,public_key)
        
    def exchange(self,sk,peer_public_key):
        shared_key = sk * peer_public_key
        return shared_key

### Teste ao esquema ECDH

In [8]:
ecdh = ECDH(163)
alice = ecdh.generate_key_pair() #pair (alice_private_key,alice_public_key)
bob = ecdh.generate_key_pair() #pair (bob_private_key,bob_public_key)
alice_shared = ecdh.exchange(alice[0],bob[1])
bob_shared = ecdh.exchange(bob[0],alice[1])

if alice_shared == bob_shared:
    print 'OK'
else:
    print 'Not OK'

OK


## Conclusão

## Referências